351 Assignment 2
James Sanii
student id: 20111876
net id: 17jcs9

This first block of code imports the data from the train and test csv files and then loops to create the training image data from the photots provided. 

In [1]:
import cv2
import pandas as pd
import numpy as np

from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import os
import random
import gc
import tensorflow as tf

train_dir = 'train (2).csv'
test_dir = 'test (2).csv'

train = pd.read_csv(train_dir,sep='\t')
test = pd.read_csv(test_dir,sep='\t')
#print(train)
temp_train_imgs = train['imageid']
temp_test_imgs = test['imageid']

mystring = '.jpg'
train_imgs = [str(s) + mystring for s in temp_train_imgs]
test_imgs = [str(s) + mystring for s in temp_test_imgs]

X = [] #images

for image in train_imgs:
    #read in each image as greyscale
    X.append(cv2.resize(cv2.imread('images/' +image), (60, 80), interpolation = cv2.INTER_CUBIC))

This creates the one hot encoding used for the output variable for the classes

In [2]:
Y = pd.get_dummies(train.label)
print(Y)

       Bags  Bottomwear  Eyewear  Fragrance  Innerwear  Jewellery  Makeup  \
0         1           0        0          0          0          0       0   
1         0           0        0          0          0          0       0   
2         0           0        0          0          0          0       0   
3         0           0        0          0          0          0       0   
4         0           1        0          0          0          0       0   
...     ...         ...      ...        ...        ...        ...     ...   
40436     0           0        0          0          0          0       0   
40437     0           0        0          0          0          0       0   
40438     0           1        0          0          0          0       0   
40439     0           0        0          0          0          0       0   
40440     0           0        0          0          0          0       0   

       Others  Sandal  Shoes  Topwear  Wallets  Watches  
0           0    

Convert the X and Y variables to numpy arrays then normalize X
Print out the shapes of the two variable to confirm they are correct

In [3]:
X = np.array(X)
Y = np.array(Y)
#normalize X values between 0 and 1
X = np.true_divide(X, 255)
print(X.shape)
print(Y.shape)

(40441, 80, 60, 3)
(40441, 13)


Same idea as above but for the testing data

In [4]:
#replicate above for testing data
X_test = []
for image in test_imgs:
    #read in each image as color
    X_test.append(cv2.resize(cv2.imread('images/' +image), (60, 80), interpolation = cv2.INTER_CUBIC))
    
Y_test = pd.get_dummies(test.label)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_test = np.true_divide(X_test, 255)
print(X_test.shape)
print(Y_test.shape)

(4000, 80, 60, 3)
(4000, 13)


Import the required libraries
Then I created a simple outline of a CNN as seen in the class lecture

In [10]:
from keras import layers
from keras import models
from keras import optimizers
import keras

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape =(80,60,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(13, activation='softmax'))

Now compile and fit the created model and see the output

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
history = model.fit(X, Y, epochs=20, batch_size = 64, verbose = 1)
print("Evaluate on test data")
results = model.evaluate(X_test, Y_test, batch_size=128)
print("test loss, test acc:", results)

Epoch 1/20
632/632 [==============================] - 66s 104ms/step - loss: 0.8603 - accuracy: 0.7322
Epoch 2/20
632/632 [==============================] - 66s 105ms/step - loss: 0.2472 - accuracy: 0.9247
Epoch 3/20
632/632 [==============================] - 66s 104ms/step - loss: 0.1748 - accuracy: 0.9439
Epoch 4/20
632/632 [==============================] - 66s 105ms/step - loss: 0.1343 - accuracy: 0.9590
Epoch 5/20
632/632 [==============================] - 66s 104ms/step - loss: 0.1057 - accuracy: 0.9659
Epoch 6/20
632/632 [==============================] - 66s 104ms/step - loss: 0.0882 - accuracy: 0.9722
Epoch 7/20
632/632 [==============================] - 66s 104ms/step - loss: 0.0762 - accuracy: 0.9762
Epoch 8/20
632/632 [==============================] - 66s 105ms/step - loss: 0.0650 - accuracy: 0.9786
Epoch 9/20
632/632 [==============================] - 66s 104ms/step - loss: 0.0564 - accuracy: 0.9813
Epoch 10/20
632/632 [==============================] - 67s 105ms/step - l

The model has an accuracy of 0.9837 on the training data but an accurracy of 0.9477 on the testing data.
Save the model so I can load it back in if needed.

In [7]:
#save the model
model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets


Model summary gives the format of the model as well as the other weights

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 78, 58, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2560)              0

Create the optimizer that is going to be used for the optimal model. 

In [25]:
#switch back to 0.0001 if worse results
opt = keras.optimizers.Adam(learning_rate=0.0005,
    beta_1=0.9,
    beta_2=0.999,)

Created second model.

The changes in the model are as followed.

Changed the optimizer. Looking online the optimizer that tended to have the best performance were variations of the Adam optimizer. When choosing the values for the Adam optimizer I kept the two beta values constant and tested results with diffrent learning rates. I started with the default which was 0.001 and this resulted in a model with similar results to Part 1. I then tried using a learning rate 10 times smaller of 0.0001 which was too slow, increased it to 0.0003 which was still too slow and struggled to get beyond 96% on testing data. I then tried it with a learning rate of 0.0005 which gave me similar results on the training data as model 1 but had increased results on the testing data to just above 96%. No learning rate schedule function was used since Adam has built in decay so manually decreasing the learning rate over time would be unnessessary. 

I added dropout layers after each Max pooling layer to reduce overfitting. I tested values of 0.1, 0.15, and 0.2 since these were the suggest values I found online to use for dropout betwen pooling layers. Each of these values were tested with each of the learning rates that were tested. 

Originally I tested it with Normalization layers instead but this gave similar results to the original model so the idea was scrapped. Dropout layers and Normalization were not used together since using Dropout layer with normalization removes the benefit of normalization. 

With these changes the model result ended up being more accurate but I struggled to get the full model to compile before my PC gave out and the python kernel crashed.

In [29]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape =(80,60,3)))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Dropout(0.2))
model2.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Dropout(0.2))
model2.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Dropout(0.2))
model2.add(layers.Conv2D(64, (3,3), activation = 'relu'))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Dropout(0.2))
model2.add(layers.Flatten())
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.Dense(13, activation='softmax'))
model2.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
counter = 0

My solution to this was to run the model for small number of epochs then save the model. I could look at the first printed counter value to see which epoch range I was in when it crashed. I could then reload the notebook at the last saved model and continue running more epochs. Since the model was created with an optimizer built into it originally, the same optimizer continues from where it left off.

This allowed me to run the model until it converged while tracking which number the model converged at. The final upgraded model converged to around 0.9760 for accurracy and scored a 0.9622 on accurracy for the testing data which is almost 2% higher than the original model even though the model scored slightly worse on the training data.

In [12]:
#run this only when the model fails
#set counter to whatever the value was when it last failed
counter = 0
# load in model2 last save
model2 = models.load_model("model_upgrade")

In [23]:
history2 = model2.fit(X, Y, epochs=5, batch_size = 64, verbose = 1)
counter = counter + 5
print("total epoch run for is:", counter)
results = model2.evaluate(X_test, Y_test, batch_size=64)
print("test loss, test acc:", results)

Epoch 1/5
632/632 [==============================] - 74s 117ms/step - loss: 0.0703 - accuracy: 0.9757
Epoch 2/5
632/632 [==============================] - 74s 116ms/step - loss: 0.0682 - accuracy: 0.9759
Epoch 3/5
632/632 [==============================] - 74s 117ms/step - loss: 0.0690 - accuracy: 0.9756
Epoch 4/5
632/632 [==============================] - 74s 117ms/step - loss: 0.0679 - accuracy: 0.9766
Epoch 5/5
632/632 [==============================] - 74s 117ms/step - loss: 0.0682 - accuracy: 0.9759
total epoch run for is: 75
63/63 [==============================] - 2s 22ms/step - loss: 0.1595 - accuracy: 0.9622
test loss, test acc: [0.1595330387353897, 0.9622499942779541]


In [14]:
model2.save("model_upgrade")

INFO:tensorflow:Assets written to: test\assets


In [13]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 78, 58, 32)        896       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 39, 29, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 39, 29, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 37, 27, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 18, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 13, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 11, 64)       